In [ ]:
!pip install scipy seaborn

In [ ]:
!pip install openpyxl

In [ ]:
import pandas as pd
import glob
import json
from scipy.stats import gaussian_kde, skew, kurtosis
import numpy as np
from scipy.signal import find_peaks
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats, signal
from scipy.fft import rfft, rfftfreq
from scipy.interpolate import interp1d
from scipy.stats import entropy


In [ ]:
file_list = glob.glob('/content/*.xlsx')
file_list = sorted(file_list)

In [ ]:
file_list

['/content/synthetic_data_class1 (2).xlsx']

Модальность, skewness, kurtosis

In [ ]:

def calculate_mean(spectral_data):
  return np.mean(spectral_data)

def calculate_std(spectral_data):
  return np.std(spectral_data)

def calculate_modality(spectral_data):
  kde = gaussian_kde(spectral_data, bw_method=0.1)
  x = np.linspace(min(spectral_data), max(spectral_data), 1000)
  y = kde(x)
  peaks, _ = find_peaks(y)
  return len(peaks)


def calculate_skewness(spectral_data):
  skewness = pd.Series(spectral_data).skew()
  return skewness

def calculate_kurtosis(spectral_data):
  kurtosis_value = kurtosis(spectral_data)
  return kurtosis_value

def calculate_IQR(spectral_data):
  q1 = np.percentile(spectral_data, 25)  # First quartile
  q3 = np.percentile(spectral_data, 75)  # Third quartile
  if (index==33):
    print(str(q1)+" "+str(q3))
  iqr_value = q3 - q1  # Interquartile Range
  return iqr_value

def calculate_SNR1(spectral_data):
  rms_signal = np.sqrt(np.mean(np.square(spectral_data)))
  rms_noise = np.std(spectral_data)
  snr = rms_signal / rms_noise
  return snr

def calculate_SNR2(spectral_data, noise):
  aver_noise=np.average(noise)
  aver_signal=np.average(spectral_data)
  return aver_signal/aver_noise

def calculate_MaxI(spectral_data):
  max=np.max(spectral_data)
  return max

def get_Intensity_On_Wavelenght(wavelenghts, wavelenght, spectral_data):
  w_array=np.array(wavelenghts)
  for index, value in np.ndenumerate(w_array):
    if abs(float(value)-wavelenght)<0.0001:
      return spectral_data[index]
  return 1

def calculate_spectral_entropy(spectral_data):
    psd = spectral_data / np.sum(spectral_data)  # Normalize to probability distribution
    return entropy(psd)

def calculate_dominant_frequency_magnitude(spectral_data):
    fft_vals = np.fft.fft(spectral_data)
    magnitudes = np.abs(fft_vals)
    return np.max(magnitudes)



In [ ]:
characteristic_wavelengths_3channel=np.array([
    590.1740112,
    589.5755615,
    589.4558105,
    546.1279907,
    545.2479248,
    543.9893188,
    543.6113892,
    541.2146606,
    540.3302002,
    537.7988281,
    533.3536377,
    527.6098022,
])
characteristic_wavelengths_2channel=np.array([
    440.481171,
    438.363617,
    432.614502,
    430.829926,
    427.163879,
    422.709778,
    403.44339,
    388.654724,
    387.844147,
    385.975616,
    382.548187,
    374.55777,
    373.728149,
    371.982452
])
characteristic_wavelengths=characteristic_wavelengths_3channel
wavelength_dict = {
}
for i in range(len(characteristic_wavelengths)):
        wavelength_dict[characteristic_wavelengths[i]] =[]



In [ ]:
sheetname='spectras'
for file in file_list:
    modalities = []
    skewness_values = []
    kurtosis_values = []
    iqr_values = []
    snr1_values=[]
    snr2_values=[]
    max_values=[]
    noise=[]
    sumI=[]
    mean=[]
    std=[]
    entropys=[]
    magnitudes=[]
    for i in range(len(characteristic_wavelengths)):
        wavelength_dict[characteristic_wavelengths[i]] =[]
    df = pd.read_excel(file, engine='openpyxl', sheet_name=sheetname)
    wavelenghts = df.columns.tolist()
    for index, row in df.iterrows():
      spectral_data = row.values  # All values in the row
      if (index==0):
        noise=np.copy(spectral_data)

      #Mean
      current_mean=calculate_mean(spectral_data)
      mean.append(current_mean)

      #STD
      current_std=calculate_std(spectral_data)
      std.append(current_std)

      #SumI
      current_sum = np.sum(spectral_data)
      sumI.append(current_sum)


      current_entropy=calculate_spectral_entropy(spectral_data)
      entropys.append(current_entropy)


      current_magnitude=calculate_dominant_frequency_magnitude(spectral_data)
      magnitudes.append(current_magnitude)




      #Modality
      modality=calculate_modality(spectral_data)
      modalities.append(modality)



      #Skewness
      skewness = calculate_skewness(spectral_data)
      skewness_values.append(skewness)

      #Kurtosis
      kurtosis_value = calculate_kurtosis(spectral_data)
      kurtosis_values.append(kurtosis_value)

      #IQR
      iqr_value=calculate_IQR(spectral_data)
      iqr_values.append(iqr_value)

      #SNR1
      snr1_value=calculate_SNR1(spectral_data)
      snr1_values.append(snr1_value)

      #SNR2
      snr2_value=calculate_SNR2(spectral_data, noise)
      snr2_values.append(snr2_value)

      #MaxI
      max_value=calculate_MaxI(spectral_data)
      max_values.append(max_value)

      for i in range(len(characteristic_wavelengths)):
        wavelength_dict[characteristic_wavelengths[i]].append(get_Intensity_On_Wavelenght(wavelenghts, characteristic_wavelengths[i], spectral_data)
)


    df['Mean']=mean
    df['STD']=std
    df['Modality'] = modalities
    df['Skewness'] = skewness_values
    df['Kurtosis'] = kurtosis_values
    df['IQR'] = iqr_values
    df['SNR1']=snr1_values
    df['SNR2']=snr2_values
    df['MaxI']=max_values
    df['SumI']=sumI
    df['Entropy']=entropys
    df['Freq_Magnitude']=magnitudes

    for i in range(len(characteristic_wavelengths)):
        df[str(characteristic_wavelengths[i])]=wavelength_dict[characteristic_wavelengths[i]]

    df.to_excel(file, index=False, sheet_name='features')

4090.78 6113.47


In [ ]:
# Create a zip file containing all .xlsx files in /content
!zip /content/all_excels.zip /content/*.xlsx

from google.colab import files
files.download('/content/all_excels.zip')

  adding: content/1.xlsx (deflated 1%)
  adding: content/synt_102.xlsx (deflated 1%)
  adding: content/synt_22.xlsx (deflated 1%)
  adding: content/synt_42.xlsx (deflated 1%)
  adding: content/synt_62.xlsx (deflated 1%)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>